In [19]:
import numpy as np
import plotly.graph_objects as go
from RRT3DstarSimple import pathPlanning as pps
from RRT3DstarObstacles import pathPlanning as ppo
from RRT3DstarObstaclesSafe import pathPlanning as ppos
from RRT3DstarMaxAngle import pathPlanning as ppm
from RRT3Dstar import pathPlanning as ppf
from TrajectorySmoothing import gradient_descent_smoothing

############################# run ########################################

start = np.array([-400, 400, 0])  # Start location in 3D
goal = np.array([400, -400, 0])  # Goal location in 3D
obstacles = [ 
    np.array([-15, 450, 50, 30, 100, 50]),
    np.array([-15, 210, 45, 30, 370, 90]),
    np.array([-15, 210, 155, 30, 370, 90]),
    np.array([-150, 15, 100, 300, 30, 200]),
    np.array([-280, -100, 100, 30, 200, 200]),
    np.array([-385, -220, 100, 235, 30, 200]),
    np.array([200, 200, 100, 100, 100, 200]),
    np.array([400, 0, 100, 100, 100, 200]),
    np.array([200, -200, 100, 100, 100, 200]),
    np.array([0, -400, 100, 100, 100, 200]),
]  # cuboids parametrized by [x, y, z, dx, dy, dz]
path1, _ = pps(start, goal, obstacles)
path2, _ = ppo(start, goal, obstacles)
path3, _ = ppm(start, goal, obstacles)
path4, _ = ppf(start, goal, obstacles)
path5, _ = ppos(start, goal, obstacles)

Path planning finished.
Simple RRT
Total nodes generated: 217
Total path length: 1476.2089623254176
Number of nodes in path: 61
Path planning finished.
RRT with Obstacle Avoidance
Total nodes generated: 284
Total path length: 1701.6782208033433
Number of nodes in path: 70
Path planning finished.
RRT with Maximum Angle
Total nodes generated: 372
Total path length: 1490.210441509718
Number of nodes in path: 61
Path planning finished.
Improved RRT
Total nodes generated: 747
Total path length: 1919.4299130894774
Number of nodes in path: 73
Path planning finished.
RRT with Obstacle Avoidance
Total nodes generated: 356
Total path length: 1715.7336621892882
Number of nodes in path: 70


In [20]:
############################################ Plotting ############################################

############################### Helper functions to plot obstacles, start, and goal ################################################################
def plot_obstacles(fig, obstacles):
    for o in obstacles:
        o = o / 100
        x = o[0] - o[3] / 2
        y = o[1] - o[4] / 2
        z = o[2] - o[5] / 2
        dx = o[3]
        dy = o[4]
        dz = o[5]
        fig.add_trace(go.Mesh3d(
            x=[x, x + dx, x + dx, x, x, x + dx, x + dx, x],
            y=[y, y, y + dy, y + dy, y, y, y + dy, y + dy],
            z=[z, z, z, z, z + dz, z + dz, z + dz, z + dz],
            color='olive',
            opacity=0.2,
            alphahull=0
        ))

def plot_start_goal(fig, start, goal):
    fig.add_trace(go.Scatter3d(
        x=[start[0]], y=[start[1]], z=[start[2]],
        mode='markers',
        marker=dict(size=8, color='blue'),
        name='Start'
    ))
    fig.add_trace(go.Scatter3d(
        x=[goal[0]], y=[goal[1]], z=[goal[2]],
        mode='markers',
        marker=dict(size=8, color='red'),
        name='Goal'
    ))

###################################### results ##########################################################################
def plotting3D(start, goal, obstacles, path1, path2, path3, path4, path5):
    """Plot the generated RRT* tree and the final path using Plotly"""
    
    #################### convert to meters ############################
    start = start / 100
    goal = goal / 100
    path1, path2, path3, path4 = path1 / 100, path2 / 100, path3 / 100, path4 / 100

    # Define figure size (in pixels)
    fig_width = 700  # Width of the figure
    fig_height = 600  # Height of the figure

    #################### Create the figure 1 ##########################################################################
    fig1 = go.Figure()
    fig1.update_layout(
    scene=dict(
        xaxis=dict(title='X Position (m)', range=[-5, 5]),
        yaxis=dict(title='Y Position (m)', range=[-5, 5]),
        zaxis=dict(title='Z Position (m)', range=[0, 2])
    ),
    title={
            'text': "3D RRT* Path Planning with Obstacle Avoidance",
            'y':0.75,  # Adjust this value to move the title vertically
            'x':0.5,  # Center the title
            'xanchor': 'center',
            'yanchor': 'top'
        },
        legend=dict(
            x=0.75,  # Move the legend closer to the plot
            y=0.65,
            xanchor='left',
            yanchor='top'
        ),
    showlegend=True,
    width=fig_width,  # Set figure width
    height=fig_height  # Set figure height
    )

    # Plot obstacles, start, goal and paths in the first figure 
    plot_obstacles(fig1, obstacles)
    plot_start_goal(fig1, start, goal)

    fig1.add_trace(go.Scatter3d(
        x=path1[:, 0], y=path1[:, 1], z=path1[:, 2],
        mode='lines',
        line=dict(color='red', width=4),
        name='RRT* Simple'
    ))
    fig1.add_trace(go.Scatter3d(
        x=path2[:, 0], y=path2[:, 1], z=path2[:, 2],
        mode='lines',
        line=dict(color='blue', width=4),
        name='RRT* with Obstacle Avoidance'
    ))
    fig1.add_trace(go.Scatter3d(
        x=path5[:, 0], y=path5[:, 1], z=path5[:, 2],
        mode='lines',
        line=dict(color='green', width=4),
        name='RRT* with Safe Obstacle Avoidance'
    ))
    fig1.show()

    ########################################### Create figure 2 #######################################
    fig2 = go.Figure()
    fig2.update_layout(
    scene=dict(
        xaxis=dict(title='X Position (m)', range=[-5, 5]),
        yaxis=dict(title='Y Position (m)', range=[-5, 5]),
        zaxis=dict(title='Z Position (m)', range=[0, 2])
    ),
    title={
        'text': "Improved 3D RRT* Path Planning",
        'y':0.75,  # Adjust this value to move the title vertically
        'x':0.5,  # Center the title
        'xanchor': 'center',
        'yanchor': 'top'
    },
    legend=dict(
        x=0.75,  # Move the legend closer to the plot
        y=0.65,
        xanchor='left',
        yanchor='top'
    ),
    showlegend=True,
    width=fig_width,  # Set figure width
    height=fig_height  # Set figure height
    )

    # Plot obstacles, start, goal and paths in the second figure
    plot_obstacles(fig2, obstacles)
    plot_start_goal(fig2, start, goal)

    fig2.add_trace(go.Scatter3d(
        x=path3[:, 0], y=path3[:, 1], z=path3[:, 2],
        mode='lines',
        line=dict(color='red', width=4),
        name='RRT* with Turning Constraints'
    ))
    fig2.add_trace(go.Scatter3d(
        x=path4[:, 0], y=path4[:, 1], z=path4[:, 2],
        mode='lines',
        line=dict(color='blue', width=4),
        name='RRT* with Ground Clearance'
    ))
    fig2.show()

plotting3D(start, goal, obstacles, gradient_descent_smoothing(path1), gradient_descent_smoothing(path2), gradient_descent_smoothing(path3), gradient_descent_smoothing(path4), gradient_descent_smoothing(path5))

Converged after 998 iterations
